Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (150000, 28, 28) (150000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (150000, 784) (150000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
num_hid_unit = 1024
beta = 0.01

graph_2 = tf.Graph()
with graph_2.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weight_w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hid_unit]))
  biases_b1 = tf.Variable(tf.zeros([num_hid_unit]))
  
  
  weight_w2 = tf.Variable(
    tf.truncated_normal([num_hid_unit, num_labels]))
  biases_b2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits_L1 = tf.matmul(tf_train_dataset, weight_w1) + biases_b1
  actFun = tf.nn.relu(logits_L1)
  
  
  logits_L2 = tf.matmul(actFun, weight_w2) + biases_b2

  # Normal loss function
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits_L2, labels=tf_train_labels))
  
  # Loss function with L2 Regularization with beta=0.01
  regularizers = tf.nn.l2_loss(weight_w1) + tf.nn.l2_loss(weight_w2)
  loss = tf.reduce_mean(loss + beta * regularizers)    

  # Optimizer. (better test accuracy with learning_rate < 0.5 (here 0.1))
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_L2)
  
  
  valid_actFun = tf.nn.relu(tf.matmul(tf_valid_dataset, weight_w1) + biases_b1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_actFun, weight_w2) + biases_b2)
  
  
  test_actFun = tf.nn.relu(tf.matmul(tf_test_dataset, weight_w1) + biases_b1)
  test_prediction = tf.nn.softmax(tf.matmul(test_actFun, weight_w2) + biases_b2)

In [6]:
num_steps = 3001

with tf.Session(graph=graph_2) as session:
  tf.global_variables_initializer().run()  #WARNING:'tf.initialize_all_variables()' is deprecated. Use `tf.global_variables_initializer()` instead
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3472.993652
Minibatch accuracy: 10.9%
Validation accuracy: 26.2%
Minibatch loss at step 500: 1152.560303
Minibatch accuracy: 82.8%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 422.138885
Minibatch accuracy: 84.4%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 154.631607
Minibatch accuracy: 85.9%
Validation accuracy: 83.9%
Minibatch loss at step 2000: 57.124851
Minibatch accuracy: 89.8%
Validation accuracy: 85.6%
Minibatch loss at step 2500: 21.428410
Minibatch accuracy: 87.5%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 8.368077
Minibatch accuracy: 86.7%
Validation accuracy: 85.2%
Test accuracy: 91.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
num_steps = 3001
restricted_dataset = train_dataset[:1000]
restricted_labels = train_labels[:1000]

with tf.Session(graph=graph_2) as session:
  tf.global_variables_initializer().run()  #WARNING:'tf.initialize_all_variables()' is deprecated. Use `tf.global_variables_initializer()` instead
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (restricted_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = restricted_dataset[offset:(offset + batch_size), :]
    batch_labels = restricted_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3564.645020
Minibatch accuracy: 7.0%
Validation accuracy: 23.4%
Minibatch loss at step 500: 1156.794678
Minibatch accuracy: 100.0%
Validation accuracy: 72.3%
Minibatch loss at step 1000: 425.348816
Minibatch accuracy: 100.0%
Validation accuracy: 72.4%
Minibatch loss at step 1500: 156.404205
Minibatch accuracy: 100.0%
Validation accuracy: 72.7%
Minibatch loss at step 2000: 57.560257
Minibatch accuracy: 100.0%
Validation accuracy: 75.0%
Minibatch loss at step 2500: 21.329515
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 3000: 8.038186
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Test accuracy: 86.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
batch_size = 128
num_hid_unit = 1024
beta = 0.01

graph_2 = tf.Graph()
with graph_2.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weight_w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hid_unit]))
  biases_b1 = tf.Variable(tf.zeros([num_hid_unit]))
  
  
  weight_w2 = tf.Variable(
    tf.truncated_normal([num_hid_unit, num_labels]))
  biases_b2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits_L1 = tf.matmul(tf_train_dataset, weight_w1) + biases_b1
  actFun = tf.nn.relu(logits_L1)
  
  # Dropout on hidden layer: RELU layer
  keep_prob = tf.placeholder("float")
  relu_layer_dropout = tf.nn.dropout(actFun, keep_prob)
    
  logits_L2 = tf.matmul(relu_layer_dropout, weight_w2) + biases_b2

  # Normal loss function
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits_L2, labels=tf_train_labels))
  
  # Loss function with L2 Regularization with beta=0.01
  regularizers = tf.nn.l2_loss(weight_w1) + tf.nn.l2_loss(weight_w2)
  loss = tf.reduce_mean(loss + beta * regularizers)    

  # Optimizer. (better test accuracy with learning_rate < 0.5 (here 0.1))
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_L2)
  
  
  valid_actFun = tf.nn.relu(tf.matmul(tf_valid_dataset, weight_w1) + biases_b1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_actFun, weight_w2) + biases_b2)
  
  
  test_actFun = tf.nn.relu(tf.matmul(tf_test_dataset, weight_w1) + biases_b1)
  test_prediction = tf.nn.softmax(tf.matmul(test_actFun, weight_w2) + biases_b2)

In [9]:
num_steps = 3001

with tf.Session(graph=graph_2) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 3677.1550293
Minibatch accuracy: 8.6
Validation accuracy: 24.8
Minibatch loss at step 500: 21.3272190094
Minibatch accuracy: 83.6
Validation accuracy: 83.7
Minibatch loss at step 1000: 0.901423096657
Minibatch accuracy: 89.1
Validation accuracy: 82.9
Minibatch loss at step 1500: 0.731554389
Minibatch accuracy: 86.7
Validation accuracy: 82.8
Minibatch loss at step 2000: 0.754748165607
Minibatch accuracy: 83.6
Validation accuracy: 82.6
Minibatch loss at step 2500: 0.797829627991
Minibatch accuracy: 82.0
Validation accuracy: 83.0
Minibatch loss at step 3000: 0.819664239883
Minibatch accuracy: 81.2
Validation accuracy: 82.6
Test accuracy: 89.8


## Extreme Overfitting

In [10]:
num_steps = 3001
restricted_dataset = train_dataset[:1000]
restricted_labels = train_labels[:1000]

with tf.Session(graph=graph_2) as session:
  tf.global_variables_initializer().run()  #WARNING:'tf.initialize_all_variables()' is deprecated. Use `tf.global_variables_initializer()` instead
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (restricted_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = restricted_dataset[offset:(offset + batch_size), :]
    batch_labels = restricted_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,  keep_prob : 0.5}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3614.366699
Minibatch accuracy: 7.8%
Validation accuracy: 27.0%
Minibatch loss at step 500: 21.250965
Minibatch accuracy: 99.2%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 0.602731
Minibatch accuracy: 100.0%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 0.422476
Minibatch accuracy: 99.2%
Validation accuracy: 79.9%
Minibatch loss at step 2000: 0.409808
Minibatch accuracy: 99.2%
Validation accuracy: 80.4%
Minibatch loss at step 2500: 0.454125
Minibatch accuracy: 98.4%
Validation accuracy: 80.2%
Minibatch loss at step 3000: 0.395022
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Test accuracy: 87.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [11]:
import math as math

In [12]:
batch_size = 128
beta = 0.001
full_img_size = image_size * image_size

hidden_nodes_1 = 1024
hidden_nodes_2 = int(hidden_nodes_1 * 0.5)
hidden_nodes_3 = int(hidden_nodes_1 * np.power(0.5, 2))
hidden_nodes_4 = int(hidden_nodes_1 * np.power(0.5, 3))
hidden_nodes_5 = int(hidden_nodes_1 * np.power(0.5, 4))

graph = tf.Graph()
with graph.as_default():

    '''Input Data'''
    # For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    '''Variables'''    
    def w_and_b(x,y):
        weights = tf.Variable(tf.truncated_normal([x, y], stddev=math.sqrt(2.0/(x))))
        biases = tf.Variable(tf.zeros([y]))
        return weights, biases
    
    # Hidden RELU layer 1
    weights_1,biases_1 = w_and_b(full_img_size,hidden_nodes_1)
    # Hidden RELU layer 2
    weights_2,biases_2 = w_and_b(hidden_nodes_1,hidden_nodes_2)
    # Hidden RELU layer 3
    weights_3,biases_3 = w_and_b(hidden_nodes_2,hidden_nodes_3)
    # Hidden RELU layer 4
    weights_4,biases_4 = w_and_b(hidden_nodes_3,hidden_nodes_4)
    # Hidden RELU layer 5
    weights_5,biases_5 = w_and_b(hidden_nodes_4,hidden_nodes_5)
    # Hidden RELU layer 6
    weights_6,biases_6 = w_and_b(hidden_nodes_5,num_labels)
    
    
    '''Training computation'''
    
    # Hidden RELU layer 1
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden_layer_1 = tf.nn.relu(logits_1)
    
    # Dropout on hidden layer: RELU layer
    keep_prob = tf.placeholder("float")
    hidden_layer_1_dropout = tf.nn.dropout(hidden_layer_1, keep_prob)
    
    def hl_dropout(x,weight,bias):
        logits = tf.matmul(x, weight) + bias
        hidden_layer = tf.nn.relu(logits)
        # Dropout on hidden layer: RELU layer
        hidden_layer_dropout = tf.nn.dropout(hidden_layer, keep_prob)
        return hidden_layer_dropout
    
    # Dropout on hidden layer 2: RELU layer
    hl2_dropout = hl_dropout(hidden_layer_1_dropout,weights_2,biases_2)
    # Dropout on hidden layer 3: RELU layer
    hl3_dropout = hl_dropout(hl2_dropout,weights_3,biases_3)
    # Dropout on hidden layer 4: RELU layer
    hl4_dropout = hl_dropout(hl3_dropout,weights_4,biases_4)
    # Dropout on hidden layer 5: RELU layer
    hl5_dropout = hl_dropout(hl4_dropout,weights_5,biases_5)
    # Output layer
    logits_6 = tf.matmul(hl5_dropout, weights_6) + biases_6 
    
    # Normal loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_6, labels=tf_train_labels))
    # Loss function with L2 Regularization with decaying learning rate beta=0.5
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + \
                   tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4) + \
                   tf.nn.l2_loss(weights_5) + tf.nn.l2_loss(weights_6)
    loss = tf.reduce_mean(loss + beta * regularizers)

    '''Optimizer'''
    # Decaying learning rate
    global_step = tf.Variable(0)  # count the number of steps taken.
    start_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_6)
    
    # Predictions for validation 
    def validation(x,weight,bias):
        valid_logits = tf.matmul(x, weight) + bias
        valid_relu = tf.nn.relu(valid_logits)
        return valid_logits, valid_relu
    
    valid_logits_1,valid_relu_1 = validation(tf_valid_dataset,weights_1,biases_1)
    valid_logits_2,valid_relu_2 = validation(valid_relu_1,weights_2,biases_2)
    valid_logits_3,valid_relu_3 = validation(valid_relu_2,weights_3,biases_3)
    valid_logits_4,valid_relu_4 = validation(valid_relu_3,weights_4,biases_4)
    valid_logits_5,valid_relu_5 = validation(valid_relu_4,weights_5,biases_5)
    valid_logits_6 = tf.matmul(valid_relu_5, weights_6) + biases_6
    
    valid_prediction = tf.nn.softmax(valid_logits_6)
    
    # Predictions for test
    
    def test(data,weights,biases):
        test_logits = tf.matmul(data, weights) + biases
        test_relu = tf.nn.relu(test_logits)
        return test_logits,test_relu
        
    test_logits_1,test_relu_1 = test(tf_test_dataset, weights_1,biases_1)
    test_logits_2,test_relu_2 = test(test_relu_1, weights_2,biases_2)
    test_logits_3,test_relu_3 = test(test_relu_2, weights_3,biases_3)
    test_logits_4,test_relu_4 = test(test_relu_3, weights_4,biases_4)
    test_logits_5,test_relu_5 = test(test_relu_4, weights_5,biases_5)
    test_logits_6 = tf.matmul(test_relu_5, weights_6) + biases_6
    
    test_prediction = tf.nn.softmax(test_logits_6)

In [13]:
num_steps = 15000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 5.21408748627
Minibatch accuracy: 8.6
Validation accuracy: 7.9
Minibatch loss at step 500: 1.64994311333
Minibatch accuracy: 83.6
Validation accuracy: 82.6
Minibatch loss at step 1000: 1.28713750839
Minibatch accuracy: 85.9
Validation accuracy: 83.7
Minibatch loss at step 1500: 1.20263493061
Minibatch accuracy: 83.6
Validation accuracy: 84.5
Minibatch loss at step 2000: 1.00675320625
Minibatch accuracy: 82.0
Validation accuracy: 84.1
Minibatch loss at step 2500: 0.924018502235
Minibatch accuracy: 85.9
Validation accuracy: 85.3
Minibatch loss at step 3000: 0.97226780653
Minibatch accuracy: 83.6
Validation accuracy: 84.6
Minibatch loss at step 3500: 0.82055079937
Minibatch accuracy: 86.7
Validation accuracy: 85.2
Minibatch loss at step 4000: 0.959862709045
Minibatch accuracy: 79.7
Validation accuracy: 85.6
Minibatch loss at step 4500: 0.863506913185
Minibatch accuracy: 81.2
Validation accuracy: 85.2
Minibatch loss at step 5000: 0.817737579346
Minibat